In [2]:
from pyspark import SparkContext

In [4]:
sc = SparkContext.getOrCreate()

In [6]:
%%writefile example2.txt
first
second line
the third line
then a fourth line

Overwriting example2.txt


In [7]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [10]:
text_rdd=sc.textFile('example2.txt')

In [14]:
words=text_rdd.map(lambda line: line.split())

In [16]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [18]:
text_rdd.collect()

['first', 'second line', 'the third line', 'then a fourth line']

In [22]:
text_rdd.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [24]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Overwriting services.txt


In [25]:
services = sc.textFile('services.txt')

In [27]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [28]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [32]:
clean=services.map(lambda line: line[1:] if line[0]=='#' else line)

In [33]:
clean = clean.map(lambda line: line.split())

In [34]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [39]:
pairs=clean.map(lambda lst: (lst[3],lst[-1]))

In [44]:
rekey=pairs.reduceByKey(lambda amt1,amt2 : float(amt1)+float(amt2))

In [45]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [46]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [50]:
# Grab State, Amount
step1 = clean.map(lambda lst: (lst[3],lst[-1]))

# Reduce by key
step2 = step1.reduceByKey(lambda amt1, amt2 : float(amt1)+float(amt2))

# Get rid of State, Amount titles
step3 = step2.filter(lambda x: not x[0]=='State')

# Sort Results by Amount
step4 = step3.sortBy(lambda stAmount: stAmount[1],ascending=False)

# Action
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]